<a href="https://colab.research.google.com/github/jlamorar/proyecto-integrador-III_tarea_2/blob/main/Customer_Segmentation_Online_retaill_nov__11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

#**Customer Segmentation - Online retail Project**

# **Phase 1: Business Understanding** Cross-Industry Standard Process for Data Mining(CRISP DM)

In ecommerce companies like online retails, customer segmentation is necessary in order to understand customers behaviors. It leverages aqcuired customer data like the one we have in our case, **transactions data** in order to divide customers into groups.

Our goal in this Notebook is to cluster our customers to get insights in:
- Increasing **revenue** (Knowing customers who present most of our revenue)
- Increasing customer **retention**
- Discovering **Trends and patterns**
- Defining **customers at risk**

We will do **RFM Analysis** as a first step and then **combine RFM with predictive algorithms (k-means)**.

RFM Analysis answers these questions:
- Who are our best customers?
- Who has the potential to be converted in more profitable customers?
- Which customers we must retain?
- Which group of customers is most likely to respond to our current campaign?

More about RFM [here](https://www.putler.com/rfm-analysis/).

# Importing main libraries

#Este bloque importa las bibliotecas necesarias para manipular datos (pandas, numpy), gestionar fechas (datetime), y aplicar modelos de clustering y métricas de evaluación (sklearn).
Las bibliotecas de visualización (matplotlib, seaborn) se utilizan para explorar y presentar gráficas que respalden los análisis.
La línea %matplotlib inline asegura que las gráficas se rendericen directamente en el notebook, y warnings.filterwarnings("ignore") elimina las advertencias para que el output sea más limpio.
Con estas bibliotecas, podemos realizar un análisis completo que incluye la limpieza de datos, el análisis exploratorio y la aplicación de modelos predictivos, respaldado por visualizaciones claras.

In [ ]:
#!pip install -U scikit-learn
#!pip install GMM
#!pip install shapeGMM

import pandas as pd
import numpy as np

import time, warnings
import datetime as dt

#modules for predictive models
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
#from sklearn.mixture import GMM

from sklearn.metrics import silhouette_samples, silhouette_score

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

# **Step 2: Data Understanding** Cross-Industry Standard Process for Data Mining(CRISP DM)

# Getting and loading the Dataset

#Aquí se carga el dataset Online Retail.xlsx en un DataFrame de pandas para analizarlo.
head() muestra las primeras 5 filas del dataset, lo que permite obtener una vista preliminar de las columnas y los datos.
Vemos las primeras filas del dataset, confirmando que contiene columnas clave como InvoiceNo, CustomerID, Quantity, UnitPrice, entre otras. Esto nos ayudará a identificar las columnas relevantes para el análisis.


In [ ]:
# Get the dataset from https://www.kaggle.com/datasets/puneetbhaya/online-retail?resource=download
#loading the dataset
retail_df = pd.read_excel("Online Retail.xlsx")

retail_df.head()

# Check the Data types

#dtypes muestra los tipos de datos de cada columna. Esto es importante para verificar si los datos están en el formato correcto antes de procesarlos.
drop(['StockCode'], axis=1) elimina una columna innecesaria que no aporta valor al análisis.
Confirmarás que CustomerID es de tipo float64, lo cual es incorrecto, y deberá ser convertido a int. Además, la columna StockCode se eliminará, reduciendo el ruido en el dataset.


In [ ]:
retail_df.dtypes
#CustomerID is float it must be changed to int datatype
#InvoiceNo must be retired from the dataframe

## Eliminamos valores nulos y convertimos la columna CustomerID a tipo entero

In [ ]:
retail_df = retail_df.dropna(subset=['CustomerID'])
retail_df['CustomerID'] = retail_df['CustomerID'].astype(int)


In [ ]:
retail_df.dtypes

#dtypes muestra los tipos de datos de cada columna y vemos que CustomerID ahora es de tipo INT. Esto es importante para verificar si los datos están en el formato correcto antes de procesarlos.
##drop(['StockCode'], axis=1) elimina una columna innecesaria que no aporta valor al análisis.


In [ ]:
#removing StockCode
retail_df2 = retail_df.drop(['StockCode'], axis=1)


In [ ]:
#verifiying dtypes again
retail_df2.dtypes


#Automatized Exploratory Data Analysis with ydata_profiling library

#Este bloque utiliza la biblioteca ydata_profiling para generar un informe exploratorio automático del dataset.El informe incluirá métricas como valores faltantes, duplicados, distribuciones de variables y correlaciones.
##Esto generará n informe detallado que resalte problemas potenciales como valores faltantes, columnas con datos altamente correlacionados o distribuciones sesgadas, lo que nos permitirá tomar decisiones informadas en la fase de preprocesamiento.


In [ ]:
!pip install ydata-profiling


#Instalamos la biblioteca ydata_profiling para automatizar el análisis exploratorio de los datos

##La utilización de esta biblioteca nos sirve para generar un informe exploratorio automático del dataset. El informe incluirá métricas como valores faltantes, duplicados, distribuciones de variables y correlaciones,
distribuciones sesgadas. Lo qu nos  permitirá tomar decisiones informadas en la fase de preprocesamiento.

In [ ]:
!pip install ydata-profiling


#Importa la clase ProfileReport de la biblioteca ydata_profiling, que se utiliza para generar informes automatizados de análisis exploratorio de datos.
#Crea un informe exploratorio basado en el DataFrame retail_df2.
Parámetros:
retail_df2: Conjunto de datos que será analizado.
title='Explorative Analysis Report': Título personalizado para el informe.
explorative=True: Activa características avanzadas como la exploración de relaciones complejas entre variables.

In [ ]:
#Installing and using the library ydata-profiling
# Examples https://github.com/ydataai/ydata-profiling
#!pip install ydata-profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(retail_df2, title='Explorative Analisys Report', explorative=True)
profile.to_widgets()

#Automatized Exploratory Data Analysis with dtale library

#Instalamos la libreria dtale que es una herramienta poderosa para realizar análisis exploratorios de datos de forma interactiva. Una vez instalada correctamente, nos permitirá explorar los datos de manera visual e intuitiva.

**#D-Tale es una librería nos permite realizar el análisis exploratorio de datos interactivo en un navegador web. Genera visualizaciones, resúmenes estadísticos y análisis interactivos basados en un DataFrame de pandas, facilitando la inspección de los datos sin necesidad de escribir mucho código**

In [ ]:
!pip install dtale


In [ ]:
#Automatized Exploratory Data Analysis
#installing and using the library dtale
#!pip install dtale
# library https://pypi.org/project/dtale/
import dtale
from dtale.views import startup
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
dtale.show(retail_df2)

# click on the following to check the Automatized Exploratory Data Analysis Report

#NOTA: El enlace creado:  https://grwzufuf87c-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1, es una URL que apunta a un servidor local donde se ejecuta la aplicación interactiva de D-Tale. Esta URL se genera porque estás trabajando en Google Colab, que configura un servidor para ejecutar y mostrar interfaces gráficas.

# **Step 3: Data Preparation** Cross-Industry Standard Process for Data Mining(CRISP DM)

#Proceso realizado anteriormente pero en este bloque desea verificar los cambios de customerID de float64 a int&$ y la eliminación de la columna StockCode que no es util para el análisis a realizar.

In [ ]:
# removing null values in CustomerID and converting CustomerID to int datatype with .astype(int) method and
retail_df2['CustomerID']  = retail_df2['CustomerID'].fillna(0).astype(int)


In [ ]:
#verifiying CustomerID dtype
retail_df2.dtypes

# Preparing the Data from the United Kingdom to analyze them

As customer clusters may vary by geography, I’ll restrict the data to only United Kingdom customers, which contains most of our customers historical data.

##Filtra los datos para mantener solo registros del Reino Unido

In [ ]:
retail_uk = retail_df2[retail_df['Country']=='United Kingdom']
#checking the shape
# customers in United Kingdom 495478
retail_uk.shape

## Este filtro selecciona solo las filas donde la columna Quantity (cantidad de productos en la orden) es mayor que 0.En este contexto, una Quantity menor o igual a 0 indica:
##Órdenes canceladas (devueltas).
##Errores en la facturación.
##Al filtrar por Quantity > 0, se asegura que solo se analicen transacciones válidas.
##retail_uk.shape:Devuelve la forma del DataFrame después de aplicar el filtro, es decir, el número de filas y columnas restantes.

In [ ]:
#removing canceled orders from the United Kingdom
retail_uk = retail_uk[retail_uk['Quantity']>0]
retail_uk.shape

#Este bloque elimina filas donde el campo CustomerID es nulo (NA) en el DataFrame retail_uk. La presencia de valores nulos en esta columna indica transacciones que no tienen un cliente asociado, lo que puede ser irrelevante para ciertos análisis, como la segmentación de clientes.

In [ ]:
#removing instances where customerID is NA
retail_uk.dropna(subset=['CustomerID'],how='all',inplace=True)
retail_uk.shape

#Este bloque filtra el DataFrame retail_uk para restringir los datos a un periodo de tiempo específico, en este caso, a transacciones realizadas a partir del 9 de diciembre de 2010. Esto es fundamental en el análisis RFM (Recency, Frequency, Monetary), ya que es más efectivo analizar métricas basadas en períodos definidos como meses o años.



In [ ]:
#restrict the data to one full year because it's better to use a metric per Months or Years in RFM Analysis
retail_uk = retail_uk[retail_uk['InvoiceDate']>= "2010-12-09"]
retail_uk.shape

##Este bloque tiene como objetivo proporcionar un resumen básico del dataset retail_uk después de las etapas de limpieza y filtrado. Específicamente, analiza la cantidad de transacciones, clientes únicos y el porcentaje de valores nulos en CustomerID.

In [ ]:
print("Summary..")
#exploring the unique values of each attribute
print("Number of transactions: ", retail_uk['InvoiceNo'].nunique())
print("Number of customers:", retail_uk['CustomerID'].nunique() )
print("Percentage of customers NA: ", round(retail_uk['CustomerID'].isnull().sum() * 100 / len(retail_df),2),"%" )

# Data Preprocessing

# Important things about a Recency, Frecuency and Monetary Value Analysis (RFM)

RFM (**Recency, Frequency, Monetary**) analysis is a customer segmentation technique that uses past purchase **behavior** to divide customers into groups. <br> RFM helps divide customers into various categories or clusters to identify customers who are more likely to respond to promotions and also for future personalization services.
- RECENCY (R): Days since last purchase
- FREQUENCY (F): Total number of purchases
- MONETARY VALUE (M): Total money this customer spent.

We will create those 3 customer attributes for each customer.


## Obtaining the Recency (how many days ago was the customer's last purchase)

To calculate recency, we need to choose a date point from which we evaluate **how many days ago was the customer's last purchase**.

In [ ]:
#last date available in our dataset
retail_uk['InvoiceDate'].max()

The last date we have is 2011-12-09 so we will use it as reference.

In [ ]:
# Stablishing the date in python to do the RFM analysis
now = dt.date(2011,12,9)
print(now)

In [ ]:
#create a new column called date which contains the date of invoice only
retail_uk['date'] = retail_uk['InvoiceDate'].dt.date

In [ ]:
retail_uk.head()

In [ ]:
#group by customers and check last date of purchase
recency_df = retail_uk.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df.head()

In [ ]:
#creating variable Recency and calculating recency in days
recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(lambda x: (now - x).days)

In [ ]:
#checking recency_df content
recency_df.head()

In [ ]:
#drop LastPurchaseDate as we don't need it anymore
recency_df.drop('LastPurchaseDate',axis=1,inplace=True)

In [ ]:
#checking CustomerID and Recency
recency_df.head()

In [ ]:
# Number of Instances and variables in recency_df.shape
recency_df.shape

Now we have the recency attribute created. e.g: Customer with ID = 12346 did his/her last purshace 325 days ago.

#El código presentado en el bloque pasado tiene como objetivo calcular la recencia de los clientes, es decir, el número de días transcurridos desde su última compra, con base en un conjunto de datos de facturas. Primero, se define una fecha de referencia (now), que es el 9 de diciembre de 2011, para calcular la diferencia en días. Luego, se extrae únicamente la fecha de las facturas (date) y se agrupan los datos por CustomerID para encontrar la última fecha de compra de cada cliente (LastPurchaseDate). Posteriormente, se calcula la recencia creando una nueva columna Recency, que mide la diferencia en días entre la fecha de referencia y la última compra. Una vez obtenido este dato, se elimina la columna de LastPurchaseDate para simplificar el DataFrame y se verifica la estructura final del mismo, que contiene únicamente el ID del cliente y su recencia. Este análisis es fundamental en estrategias de marketing para identificar clientes recientes o inactivos y desarrollar campañas personalizadas basadas en su comportamiento de compra.

# Obtaining the Frequency (how many times a customer purchased from the Online Retail Company)

Frequency helps us to know **how many times a customer purchased from us**. To do that we need to check how many invoices are registered by the same customer.

In [ ]:
# drop duplicates CustomerIDs
retail_uk_copy = retail_uk
retail_uk_copy.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)


#Calculate frequency of purchases by CustomerID
frequency_df = retail_uk_copy.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

#Este bloque de codigo tiene como finalidad calcular la frecuencia de compras de los clientes, es decir, cuántas veces un cliente ha realizado una transacción en el conjunto de datos. Primero, se realiza una copia del DataFrame original retail_uk para trabajar sobre ella sin modificar el original. Luego, se eliminan las filas duplicadas en la combinación de las columnas InvoiceNo (número de factura) y CustomerID, manteniendo solo la primera ocurrencia. Esto asegura que cada factura de un cliente se cuente solo una vez. Posteriormente, se agrupan los datos por CustomerID y se cuenta la cantidad de facturas (InvoiceNo) asociadas a cada cliente. El resultado se almacena en un nuevo DataFrame frequency_df, que contiene dos columnas: CustomerID y Frequency, donde esta última indica cuántas transacciones ha realizado cada cliente. Este análisis es útil para medir la frecuencia de compra, un componente clave del modelo RFM (Recency, Frequency, Monetary) utilizado en la segmentación de clientes y la toma de decisiones estratégicas en marketing.

## Monetary Value (How much money a customer spent over time in the Online Retail Company)

Monetary attribute answers the question: **How much money did the customer spent over time?**

To do that, first, we will create a new column total cost to have the total price per invoice.

In [ ]:
#create column TotalCost from customers in United Kingdom
retail_uk['TotalCost'] = retail_uk['Quantity'] * retail_uk['UnitPrice']

In [ ]:
# Obtaining the sum or TotalCost of all purchases by CustomerID
monetary_df = retail_uk.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

#Este código tiene como propósito calcular el valor monetario total de las compras realizadas por cada cliente en el Reino Unido, un componente fundamental del análisis RFM.

Primero, se crea una nueva columna en el DataFrame retail_uk llamada TotalCost, que se obtiene multiplicando la cantidad de productos comprados (Quantity) por el precio unitario (UnitPrice) de cada transacción. Esto representa el costo total de cada factura.

Luego, se agrupan los datos por CustomerID utilizando la función groupby y se calcula la suma total de los valores de TotalCost para cada cliente mediante la función de agregación agg. El resultado se guarda en un nuevo DataFrame monetary_df, que contiene dos columnas: CustomerID y Monetary. La columna Monetary refleja el valor monetario total gastado por cada cliente. Finalmente, se utiliza head() para mostrar las primeras cinco filas de este DataFrame, facilitando la revisión de los resultados.

Este análisis es crucial para entender el valor económico que cada cliente aporta al negocio, permitiendo segmentar a los clientes según su valor y priorizar estrategias de retención o fidelización para los más rentables.

# Creating the Recency, Frequency and Monetary value dataframe

In [ ]:
#merge recency dataframe with frequency dataframe
temp_df = recency_df.merge(frequency_df,on='CustomerID')
temp_df.head()

In [ ]:
#merging the monetary_df dataframe temp_df
rfm_df = temp_df.merge(monetary_df,on='CustomerID')

#and using CustomerID as index in the dataframe
rfm_df.set_index('CustomerID',inplace=True)

#check the head
rfm_df.head()

#Código anterior combina los DataFrames de recencia, frecuencia y valor monetario total de las compras para construir un único DataFrame consolidado llamado rfm_df, utilizado en el análisis RFM (Recency, Frequency, Monetary). Primero, se fusionan los DataFrames de recencia (recency_df) y frecuencia (frequency_df) utilizando la columna CustomerID como clave, creando un DataFrame temporal (temp_df) que incluye las columnas Recency y Frequency. Luego, este se fusiona con el DataFrame de valor monetario (monetary_df) para incorporar la columna Monetary. Finalmente, se establece CustomerID como índice del DataFrame resultante, facilitando el acceso a los datos de cada cliente. El DataFrame final rfm_df permite analizar de manera integral el comportamiento de compra de los clientes, proporcionando métricas clave para segmentación y estrategias de marketing.

Customer with ID = 12346 has recency: 325 days, frequency:1, and monetary: 77183,60 £.

### RFM Table Correctness verification

In [ ]:
retail_uk[retail_uk['CustomerID']==12346.0]

In [ ]:
(now - dt.date(2011,1,18)).days == 325

#Estas dos lineas de codigo filtran las transacciones del cliente con CustomerID 12346.0 en el DataFrame retail_uk, mostrando todas las compras realizadas por este cliente, como fechas, cantidades y precios, para analizar su comportamiento. Posteriormente, calcula la diferencia en días entre el 9 de diciembre de 2011 (now) y el 18 de enero de 2011 (dt.date(2011,1,18)), lo que da un total de 325 días, y verifica si este valor coincide con el esperado utilizando la expresión lógica == 325. Este análisis es útil para validar cálculos de recencia en el contexto del modelo RFM y asegurar que los datos sean precisos al medir el tiempo desde la última compra de un cliente.

As we can see our RFM table is correct. The first customer bought only once, and only one product with huge amount.

## Customer segments with RFM Model

Before moving to customer segments, Let's see the application of Pareto Principle – commonly referred to as the 80-20 rule on our dataset by applying it to our RFM variables.

Pareto’s rule says **80% of the results come from 20% of the causes**.

Similarly, **20% customers contribute to 80% of your total revenue**. Let's verify that because that will help us know which customers to focus on when marketing new products.

#Verificando la ley de PARETO

### Applying 80-20 rule

In [ ]:
#get the 80% of the revenue
pareto_cutoff = rfm_df['Monetary'].sum() * 0.8
print("The 80% of total revenue is: ",round(pareto_cutoff,2))

In [ ]:
customers_rank = rfm_df
# Create a new column that is the rank of the value of coverage in ascending order
customers_rank['Rank'] = customers_rank['Monetary'].rank(ascending=0)
#customers_rank.drop('RevenueRank',axis=1,inplace=True)
customers_rank.head()

#El bloque anterior implementa un análisis basado en el principio de Pareto (80/20) para identificar a los clientes más valiosos que generan el 80% de los ingresos totales. Primero, calcula el 80% del ingreso total sumando los valores de la columna Monetary en el DataFrame rfm_df y multiplicando por 0.8, almacenando este valor en la variable pareto_cutoff. Luego, se agrega una nueva columna Rank al DataFrame customers_rank, que clasifica a los clientes según su contribución monetaria de mayor a menor (ascending=0). Este ranking facilita identificar rápidamente cuáles clientes aportan la mayor parte de los ingresos, lo que es fundamental para la segmentación y estrategias de fidelización enfocadas en los clientes más rentables.

### Top Customers

In [ ]:
customers_rank.sort_values('Rank',ascending=True)

In [ ]:
#get top 20% of the customers
top_20_cutoff = 3863 *20 /100
top_20_cutoff

In [ ]:
#sum the monetary values over the customer with rank <=773
revenueByTop20 = customers_rank[customers_rank['Rank'] <= 772]['Monetary'].sum()
revenueByTop20

#En este bloque anterior se ordena el DataFrame customers_rank por la columna Rank en orden ascendente para priorizar a los clientes con mayor contribución monetaria. Luego, calcula el número de clientes que representan el 20% superior del total, determinando que son 772 clientes de un total de 3863. Posteriormente, filtra estos clientes utilizando la columna Rank y suma sus valores monetarios (Monetary) para obtener el ingreso total generado por el top 20%, almacenándolo en la variable revenueByTop20. Este análisis confirma que una pequeña fracción de los clientes (20%) genera la mayor parte de los ingresos, alineándose con el principio de Pareto, y permite a la empresa enfocar sus esfuerzos en estos clientes clave para maximizar la rentabilidad.

In our case, the 80% of total revenue is not achieved by the 20% of TOP customers but approximately, it does, because they are less than our 20% TOP customers who achieve it. It would be interesting to study this group of customers because they are those who make our most revenue.

### Applying RFM score formula

The simplest way to create customers segments from RFM Model is to use **Quartiles**. We assign a score from 1 to 4 to Recency, Frequency and Monetary. Four is the best/highest value, and one is the lowest/worst value. A final RFM score is calculated simply by combining individual RFM score numbers.

Note: Quintiles (score from 1-5) offer better granularity, in case the business needs that but it will be more challenging to create segments since we will have 5*5*5 possible combinations. So, we will use quartiles.

#### RFM Quartiles

In [ ]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

In [ ]:
quantiles.to_dict()

#El código calcula los cuartiles (25%, 50% y 75%) de las métricas RFM (Recency, Frequency y Monetary) en el DataFrame rfm_df mediante la función quantile, lo que permite dividir los datos en cuatro partes iguales y entender la distribución de cada métrica. Luego, convierte estos cuartiles en un diccionario utilizando to_dict(), lo que facilita su uso posterior para segmentar clientes en diferentes categorías según su comportamiento de compra. Esta segmentación es clave para identificar patrones y desarrollar estrategias personalizadas para distintos grupos de clientes.

#### Creation of RFM segmentation table

We will create two segmentation classes since, high recency is bad, while high frequency and monetary value is good.

In [ ]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

In [ ]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [ ]:
rfm_segmentation.head()

#Los bloques anteriores implementan un sistema de puntuación basado en los cuartiles de las métricas RFM (Recency, Frequency y Monetary) para segmentar a los clientes. Primero, define dos funciones: RScore y FMScore. La función RScore asigna puntuaciones inversas a la recencia, donde los clientes con valores más bajos (reciente) obtienen una puntuación más alta (4 para el mejor cuartil). Por otro lado, la función FMScore asigna puntuaciones directas a la frecuencia y el valor monetario, donde los clientes con valores más altos obtienen mejores puntuaciones (4 para el mejor cuartil). Luego, se crea un nuevo DataFrame rfm_segmentation, al que se añaden tres nuevas columnas (R_Quartile, F_Quartile y M_Quartile) que contienen las puntuaciones basadas en la posición de cada cliente dentro de los cuartiles de cada métrica. Este sistema de puntuación permite clasificar a los clientes en segmentos clave según su comportamiento de compra y facilita la toma de decisiones estratégicas en marketing.








Now that we have the score of each customer, we can represent our customer segmentation.<br>
First, we need to combine the scores (R_Quartile, F_Quartile,M_Quartile) together.

In [ ]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

#El código crea una nueva columna llamada RFMScore en el DataFrame rfm_segmentation, que combina las puntuaciones individuales de Recency, Frequency y Monetary en un único valor concatenado. Utiliza map(str) para convertir las puntuaciones de cada métrica en cadenas de texto y luego las concatena, generando un puntaje como "423", donde cada dígito representa la posición del cliente en los cuartiles de las métricas correspondientes. Este puntaje resume el comportamiento del cliente en un solo indicador, facilitando su segmentación en grupos específicos según su valor para la empresa. Finalmente, se utiliza head() para verificar las primeras filas del DataFrame y confirmar que la columna se haya generado correctamente, lo que permite identificar clientes clave para estrategias de marketing personalizadas.

Best Recency score = 4: most recently purchase.
Best Frequency score = 4: most quantity purchase.
Best Monetary score = 4: spent the most.

Let's see who are our **Champions** (best customers).

In [ ]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

#Este código filtra el DataFrame rfm_segmentation para identificar a los clientes con la mejor puntuación posible de RFMScore, es decir, '444'. Esto significa que estos clientes se encuentran en el mejor cuartil para Recency (recientes), Frequency (frecuentes) y Monetary (mayor gasto). Luego, los resultados se ordenan por la columna Monetary en orden descendente, mostrando primero a los clientes que han gastado más. Finalmente, se utiliza head(10) para visualizar los 10 clientes con mayor valor monetario dentro del segmento más valioso, lo que permite identificar a los clientes más importantes en términos de ingresos y actividad reciente. Esto es clave para diseñar estrategias de fidelización enfocadas en mantener la relación con estos clientes.

We can find [here](http://www.blastam.com/blog/rfm-analysis-boosts-sales) a suggestion of key segments and then we can decide which segment to consider for further study.

**Note:** the suggested link use the opposite valuation: 1 as highest/best score and 4 is the lowest.

**How many customers do we have in each segment?**

In [ ]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

#Este código clasifica a los clientes en categorías clave según sus puntuaciones RFM y cuenta cuántos pertenecen a cada segmento. Los Best Customers (puntuación '444') son los más valiosos por su alta recencia, frecuencia y valor monetario. Los Loyal Customers (frecuencia alta) compran regularmente, mientras que los Big Spenders (valor monetario alto) generan mayores ingresos. Los Almost Lost ('244') eran clientes frecuentes y de alto valor, pero su actividad reciente ha disminuido. Los Lost Customers ('144') gastaban mucho, pero no han comprado recientemente, y los Lost Cheap Customers ('111') son los menos valiosos, con poca recencia, baja frecuencia y bajo gasto. Esta segmentación permite estrategias personalizadas para cada grupo.

Now that we knew our customers segments we can choose how to target or deal with each segment.

For example:

**Best Customers - Champions**: Reward them. They can be early adopters to new products. Suggest them "Refer a friend".

**At Risk**: Send them personalized emails to encourage them to shop.

More ideas about what actions to perform in [Ometria](http://54.73.114.30/customer-segmentation#).

### Conclusion - perspective from this level of customer segmentation
To gain even further insight into customer behavior, we can dig deeper in the relationship between RFM variables.  

RFM model can be used in conjunction with certain predictive models like **k-means clustering**, **Logistic Regression** and **Recommendation** to produce better informative results on customer behavior.

We will go for k-means since it has been widely used for Market Segmentation and it offers the advantage of being simple to implement, following Andrew Ng who advice in his Machine Learning course, start with a dirty and simple model then move to more complex models because simple implementation helps having a first glance at the data and know where/how to exploit it better.

## Applying K-means clustering on RFM variables

### Preprocess Data

In [ ]:
rfm_data = rfm_df.drop(['R_Quartile','F_Quartile','M_Quartile','RFMScore'],axis=1)
rfm_data.head()

#Este código elimina las columnas R_Quartile, F_Quartile, M_Quartile y RFMScore del DataFrame rfm_df, dejando únicamente las métricas originales de Recency, Frequency y Monetary en un nuevo DataFrame llamado rfm_data. Esto se hace para enfocarse en las variables numéricas sin incluir las clasificaciones derivadas, permitiendo un análisis más limpio y adecuado para técnicas como clustering o modelos estadísticos. Finalmente, se utiliza head() para verificar que el DataFrame resultante contenga solo las columnas requeridas.

#### Feature correlations

In [ ]:
rfm_data.corr()

In [ ]:
sns.heatmap(rfm_data.corr())

#Estas 2 líneas de codigo calculan la matriz de correlación de las métricas Recency, Frequency y Monetary en el DataFrame rfm_data utilizando corr(), lo que permite identificar la relación lineal entre estas variables. Posteriormente, se visualiza esta matriz con un mapa de calor (heatmap) de la librería seaborn, donde los colores indican la magnitud y dirección de las correlaciones. Un valor cercano a 1 o -1 muestra una fuerte correlación positiva o negativa, respectivamente, mientras que un valor cercano a 0 indica poca o ninguna relación. Este análisis ayuda a entender cómo estas métricas están relacionadas, lo que puede ser clave para ajustar estrategias de segmentación o validar la independencia de las variables para modelos predictivos.

On one hand, we have a negative correlation between:
- Recency and Frequency
- Recency and Monetary

On the other hand, the correlation between **Monetary and Frequency** is positive comparing to negative ones but still not that strong.

#### Visualize feature distributions

To get a better understanding of the dataset, we can construct a scatter matrix of each of the three features present in the RFM data.

In [ ]:
# Produce a scatter matrix for each pair of features in the data
scatter_matrix(rfm_data, alpha = 0.3, figsize = (11,5), diagonal = 'kde');

#Este codigo genera una matriz de dispersión para las métricas Recency, Frequency y Monetary en el DataFrame rfm_data utilizando la función scatter_matrix. Esta matriz muestra gráficos de dispersión para cada par de variables, permitiendo visualizar la relación entre ellas. Además, en la diagonal principal se presentan gráficos de densidad (especificados por diagonal='kde'), que muestran la distribución de cada métrica individualmente. La transparencia de los puntos se controla con el parámetro alpha=0.3, y el tamaño del gráfico se ajusta con figsize=(11,5). Este análisis visual es útil para identificar patrones, correlaciones no lineales y posibles outliers en las métricas RFM, facilitando la interpretación del comportamiento de los clientes

We can notice that we have a **skewed distribution** of the 3 variables and there exist **outliers**.

This indicates how normalization is required to make the data features normally distributed as **clustering** algorithms **require** them to be **normally distributed**.

#### Data Normalization

In [ ]:
#log transformation
rfm_r_log = np.log(rfm_data['Recency']+0.1) #can't take log(0) and so add a small number
rfm_f_log = np.log(rfm_data['Frequency'])
rfm_m_log = np.log(rfm_data['Monetary']+0.1)

In [ ]:
log_data = pd.DataFrame({'Monetary': rfm_m_log,'Recency': rfm_r_log,'Frequency': rfm_f_log})

In [ ]:
log_data.head()

#El código aplica una transformación logarítmica a las métricas Recency, Frequency y Monetary para reducir la dispersión y manejar mejor las distribuciones sesgadas. Dado que el logaritmo de cero no está definido, se suma un pequeño valor (0.1) a las columnas Recency y Monetary antes de aplicar la transformación. Esto ayuda a estabilizar la varianza y a hacer que los datos sean más adecuados para análisis estadísticos o modelos de machine learning que asumen distribuciones normales. Finalmente, se crea un nuevo DataFrame log_data con las métricas transformadas y se utiliza head() para verificar los primeros registros, lo que permite observar el efecto de la transformación en los valores.

In [ ]:
# Produce a scatter matrix for each pair of features in the data
scatter_matrix(log_data, alpha = 0.2, figsize = (11,5), diagonal = 'kde');

#Este código genera una matriz de dispersión para las métricas transformadas logarítmicamente (Recency, Frequency y Monetary) contenidas en el DataFrame log_data. Este gráfico muestra relaciones visuales entre cada par de variables, permitiendo observar si la transformación ha logrado una distribución más uniforme y lineal de los datos. En la diagonal principal, se presentan gráficos de densidad (kde), que ilustran cómo se distribuyen las métricas individuales después de la transformación. La transparencia de los puntos se ajusta con alpha=0.2 para facilitar la visualización en áreas densas, y el tamaño general del gráfico se define con figsize=(11,5). Este análisis ayuda a identificar patrones más claros y mejora la interpretación de las relaciones entre las métricas transformadas.








The distributions of Frequency and Monetary are better, more normalized, but it's not the case with Recency Distribution, which is improved but not as much.

In [ ]:
sns.heatmap(rfm_data.corr())

In [ ]:
sns.heatmap(log_data.corr())

In [ ]:
log_data.corr()

#Los 3 bloques anteriores calculan y visualizan las matrices de correlación tanto para las métricas originales (rfm_data) como para las métricas transformadas logarítmicamente (log_data) utilizando mapas de calor (heatmap) y la función corr(). Estas matrices muestran la relación lineal entre las variables Recency, Frequency y Monetary, con valores entre -1 y 1 que indican la dirección y magnitud de las correlaciones. Al comparar las matrices, se puede observar cómo la transformación logarítmica afecta las correlaciones, pudiendo suavizar relaciones no lineales y mejorar la interpretabilidad. Los mapas de calor ofrecen una representación visual clara, donde colores más oscuros indican correlaciones más fuertes. Este análisis es útil para validar la idoneidad de las transformaciones y su impacto en la relación entre las métricas.

Now, Monetary and Frequency are more strongly correlated.

### K-means Implementation

A common challenge with k-means is that you must tell it how many clusters you expect. Figuring out how many clusters we need is not obvious from data, thus we will try different clusters numbers and check their [silhouette coefficient](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html). The silhouette coefficient for a data point measures how similar it is to its assigned cluster from -1 (dissimilar) to 1 (similar). The [elbow](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set#The_elbow_method) method can be used to determine the number of clusters as well.

**Note:** K-means is sensitive to initializations because those initializations are critical to quality of optima found. Thus, we will use smart initialization called ***k-means++***.

In [ ]:
matrix = log_data.as_matrix()
for n_clusters in range(2,10):
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=100)
    kmeans.fit(matrix)
    clusters = kmeans.predict(matrix)
    silhouette_avg = silhouette_score(matrix, clusters)
    print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

#Este código utiliza el algoritmo K-Means para segmentar clientes en el conjunto de datos transformados log_data, evaluando la calidad de la segmentación mediante el silhouette score. Convierte el DataFrame en una matriz para su procesamiento y realiza iteraciones con diferentes números de clusters, desde 2 hasta 9. En cada iteración, entrena el modelo de K-Means con inicialización k-means++ y ejecuta 100 iteraciones para seleccionar el mejor resultado. Luego, predice los clusters y calcula el silhouette score, que mide la cohesión interna y separación entre clusters. Finalmente, imprime el promedio del silhouette score para cada número de clusters, ayudando a identificar el número óptimo de grupos que maximiza la calidad de la segmentación.

The **best silhouette score** obtained is when the **number of clusters is 2**.

In [ ]:
n_clusters = 2
kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30)
kmeans.fit(matrix)
clusters_customers = kmeans.predict(matrix)
silhouette_avg = silhouette_score(matrix, clusters_customers)
print('score de silhouette: {:<.3f}'.format(silhouette_avg))

# Aquí se realiza una segmentación de clientes utilizando el algoritmo K-Means con 2 clusters y evalúa la calidad de la segmentación a través del silhouette score. Se utiliza la inicialización k-means++ para optimizar la elección de centroides y se ejecuta el algoritmo 30 veces (n_init=30) para garantizar un resultado robusto. Después de ajustar el modelo a los datos, se predicen los clusters de cada cliente en la matriz transformada y se calcula el silhouette score, que mide la cohesión y separación de los clusters. Finalmente, se imprime el score con tres decimales, proporcionando una métrica que permite evaluar qué tan bien están agrupados los clientes en estos 2 clusters.

#### Visualize Clusters

In [ ]:
#create a scatter plot
plt.scatter(matrix[:, 0], matrix[:, 1], c=clusters_customers, s=50, cmap='viridis')
#select cluster centers
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

In [ ]:
# What's the number of customers in each cluster?
pd.DataFrame(pd.Series(clusters_customers).value_counts(), columns = ['NumberCustomers']).T

#Aquí se genera un diagrama de dispersión que visualiza la segmentación de clientes en dos clusters, utilizando las primeras dos métricas transformadas de la matriz log_data como ejes. Los puntos de datos están coloreados según su pertenencia a uno de los clusters, con una paleta viridis. Adicionalmente, se marcan los centroides de los clusters en negro para mostrar sus posiciones centrales, con un tamaño mayor y una transparencia ajustada para destacarlos. Finalmente, se calcula la cantidad de clientes en cada cluster mediante un conteo de las asignaciones de clusters (clusters_customers), mostrando los resultados en un DataFrame que indica el número total de clientes por cluster. Esta visualización y conteo facilitan la interpretación del tamaño y distribución de los grupos generados por el modelo K-Means.

**Note(Additional):** We can check the median of each variable (Frequency, Monetary, Recency) in each cluster in order to understand what customers does each cluster represent.

**Conclusion - Perspective after applying k-means clustering**:

Unfortunately, we didn't obtain a clearly separated clusters. Clusters assignments are muddled. (It may be due to outliers who weren't removed).

Limitations of k-means clustering:
- There is no assurance that it will lead to the ***global*** best solution.
- Can't deal with **different shapes**(not circular) and consider one point's probability of belonging to more than one cluster.

These disadvantages of k-means mean that for many datasets (especially low-dimensional datasets) it may not perform as well as you might hope. Here comes Guassian Mixture Model (GMM) in help by providing greater flexibility due to clusters having unconstrained covariances and allowing probabilistic cluster assignment.

Reference  ***Python Data Science Handbook*** by ***Jake VanderPlas***.

**Note- Further Explanation:** A common practice before doing clustering: **Principal Component Analysis (PCA)**. PCA calculates the dimensions which best maximize variance.It gives directions on how many components to consider for GMM. Basically, it does **dimensionality reduction** while keeping the most important features, characteristics (combinations of features best describe customers). But as we are not dealing with high dimension we won't do it for this case.

### Gaussian Mixture Model Implementation

While k-means is easy to understand and implement.It fails when dealing with non-circular shapes and lack of probabilistic cluster assignment—mean that for many datasets (especially low-dimensional datasets) it may not perform as well as you might hope.

In [ ]:
gmm = GMM(n_components=2).fit(matrix)
labels = gmm.predict(matrix)
plt.scatter(matrix[:, 0], matrix[:, 1], c=labels, s=40, cmap='viridis');

#Aquí seutiliza el modelo Gaussian Mixture Model (GMM) para segmentar los datos en 2 componentes y visualizar los resultados. Primero, el modelo se entrena con los datos transformados (matrix) utilizando GMM(n_components=2), que asume que los datos provienen de una mezcla de dos distribuciones gaussianas. Luego, se predicen las etiquetas de cada punto, asignándolos al componente al que tienen mayor probabilidad de pertenecer. Finalmente, se genera un gráfico de dispersión donde los puntos están coloreados según su cluster, utilizando la paleta viridis. Este método permite una segmentación más flexible que K-Means, ya que considera formas y distribuciones más complejas, proporcionando una visión probabilística de los clusters en los datos.

#ANÁLISIS DE LA ACTIVIDAD

#El siguiente paso clave es presentar los resultados del clustering a los interesados dentro de la empresa para recoger sus aportaciones y ajustar el análisis en función de sus necesidades estratégicas. Es fundamental comprender cómo la organización planea utilizar la segmentación y qué nivel de detalle desean en la clasificación de los clientes. Por ejemplo, los stakeholders podrían estar interesados en analizar un espectro completo de comportamiento, desde los clientes de mayor valor (que compran frecuentemente, gastan más y lo hacen de manera reciente) hasta los de menor valor (clientes inactivos con baja frecuencia y gasto). La retroalimentación obtenida permitirá refinar las técnicas de clustering, optimizando la segmentación de clientes para que se alinee mejor con los objetivos comerciales. Esto puede implicar trabajar con las métricas RFM o con los datos transaccionales completos para identificar patrones más complejos. Si se dispone de más tiempo, se puede llevar el análisis un paso más allá mediante la implementación de modelos de clasificación para predecir la pertenencia de futuros clientes a los clusters definidos. Esto incluye probar diferentes algoritmos, como Support Vector Classifier (SVC) y Regresión Logística, que son herramientas potentes para clasificar nuevos datos. El proceso implica dividir los datos en conjuntos de entrenamiento y prueba, entrenar cada modelo y luego evaluar su desempeño utilizando métricas como la precisión, la sensibilidad y la puntuación F1. Comparar estos resultados permitirá seleccionar el modelo que ofrezca la mejor capacidad predictiva. Este enfoque no solo ayudará a clasificar correctamente a los nuevos clientes, sino que también permitirá a la empresa anticipar comportamientos futuros y ajustar sus estrategias de marketing, retención o fidelización de manera proactiva y personalizada.

#Hemos identificado y segmentado clientes mediante el análisis RFM y técnicas de clustering (K-Means, GMM). Aunque no hemos mencionado explícitamente la tasa de deserción (churn), el análisis de recencia permite inferir clientes con alto riesgo de abandono (por ejemplo, clientes "Lost" o "Almost Lost"), lo cual forma parte del análisis de churn.

#Aunque se han identificado clientes en riesgo mediante clustering y segmentación RFM, aún faltan recomendaciones específicas para retener a los clientes que están a punto de abandonar o para mejorar la relación con otros segmentos. Para complementar esta pregunta, sería necesario:
#Proponer estrategias concretas de retención basadas en los resultados (ej. campañas personalizadas para "Almost Lost" o incentivos para "Loyal Customers").
#Analizar más detalladamente patrones de deserción con datos históricos (ej. identificación de causas específicas de churn).
#Evaluar cómo se podrían implementar estas recomendaciones en términos de impacto comercial.